### 1) This notebook focuses on detailed analysis of Primary Care Shortage Severity from various aspects:
#### i) Zip Code Level
#### ii) City Level
#### iii) Different Poverty Level
#### iv) Services Type Level

### 2) Comparison: LA vs 6 other Major Cities in County on Shortage Severity
### (How Shortage Severity can impact on Community Development/Safety)
#### i) Different Poverty Level
#### ii) Services Type Level

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.gridspec as gridspec
#la_facility_mua = pd.read_csv('../input/la-final-data-2/la_facility_mua.csv')
la_facility_pcua = pd.read_csv('../input/la-final-data-2/la_facility_pcua.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
la_facility_pcua.head(5)

,source,cat1,cat2,org_name,Name,addrln1,city,state,post_id,description,link,use_type,latitude,longitude,date_updated,Zip Code,facility_name,facility_description,facility_status,facility_number_of_beds,license_category,facility_er_service,facility_address_1,HPSA_RuralStatus,HPSA_MSSA_Name_1,HPSA_MSSA_Name_2,HPSA_Subtype,HPSA_FTE,HPSA_Score,HPSA_Designated_Time,HPSA_Updated_Time,HPSA_PctPoverty,HPSA_Shortage,HPSA_Ratio,HPSA_CountPoverty,HPSA_Pop_100Poverty,HPSA_Pop_200Poverty,HPSA_Pct_100Poverty,HPSA_Pct_200Poverty,HPSA_UnitCount
0,211,Social Services,Children and Family Services,neweconomicsforwomen.org,New Economics For Women - Familysource Center ...,501 S. Bixel St.,Los Angeles,CA,56652,The agency is an economic development organiza...,http://egis3.lacounty.gov/lms/?p=56652,publish,34.054842,-118.261689,1370112656000,90017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,211,Social Services,Children and Family Services,NaN,Shields For Families Inc - Compton Family Pres...,215 Acacia Ave.,Compton,CA,621,"The agency provides child abuse services, dome...",http://egis3.lacounty.gov/lms/?p=621,publish,33.897255,-118.226657,1370112656000,90220.0,"Bsnj Health Center, Inc.",Parent Facility,Open,19.0,Community Clinic,Not Applicable,201 N Central Ave,Urban,Athens/Gardena Northeast/Harbor Gateway North,MSSA 78.2UUU/Athens,Hpsa Geographic High Needs,3.75,18.0,1.349741e+12,1.514333e+12,22.4,24.58,22665:1,82904.0,18583.0,43222.0,22.415083,52.135000,21.0
2,211,Social Services,Children and Family Services,NaN,Shields For Families Inc - Compton Family Pres...,215 Acacia Ave.,Compton,CA,621,"The agency provides child abuse services, dome...",http://egis3.lacounty.gov/lms/?p=621,publish,33.897255,-118.226657,1370112656000,90220.0,Gateway Plaza Dialysis,Parent Facility,Open,19.0,Chronic Dialysis Clinic,Not Applicable,1580 W Rosecrans Ave,Urban,Athens/Gardena Northeast/Harbor Gateway North,MSSA 78.2UUU/Athens,Hpsa Geographic High Needs,3.75,18.0,1.349741e+12,1.514333e+12,22.4,24.58,22665:1,82904.0,18583.0,43222.0,22.415083,52.135000,21.0
3,211,Social Services,Children and Family Services,NaN,Shields For Families Inc - Compton Family Pres...,215 Acacia Ave.,Compton,CA,621,"The agency provides child abuse services, dome...",http://egis3.lacounty.gov/lms/?p=621,publish,33.897255,-118.226657,1370112656000,90220.0,Aghaby Comprehensive Community Health Center,Parent Facility,Open,19.0,Community Clinic,Not Applicable,349 W Compton Blvd,Urban,Compton East,MSSA 78.2BBB/Compton East,Hpsa Geographic High Needs,8.58,18.0,1.395360e+12,1.514333e+12,29.0,21.85,10640:1,90233.0,25368.0,56536.0,28.113883,62.655569,20.0
4,211,Social Services,Children and Family Services,NaN,Shields For Families Inc - Compton Family Pres...,215 Acacia Ave.,Compton,CA,621,"The agency provides child abuse services, dome...",http://egis3.lacounty.gov/lms/?p=621,publish,33.897255,-118.226657,1370112656000,90220.0,Compton Community Hemodialysis Center,Parent Facility,Open,19.0,Chronic Dialysis Clinic,Not Applicable,801 W Compton Blvd,Urban,Compton East,MSSA 78.2BBB/Compton East,Hpsa Geographic High Needs,8.58,18.0,1.395360e+12,1.514333e+12,29.0,21.85,10640:1,90233.0,25368.0,56536.0,28.113883,62.655569,20.0


### A) Data Cleaning
#### i) Drop useless columns (with only one unique value)
#### ii) Fill Null values mode/median
#### iii) Correct individual values in some features

In [4]:
#la_facility_mua.drop(['use_type','facility_number_of_beds','state'], axis=1, inplace=True)
la_facility_pcua.drop(['use_type','facility_number_of_beds','state'], axis=1, inplace=True)

In [5]:
la_facility_pcua['source'].fillna('LA County Department of Children and Family Services', inplace=True)

la_facility_pcua['HPSA_RuralStatus'].fillna('Rural', inplace=True)
la_facility_pcua['HPSA_Subtype'].fillna('Hpsa Population', inplace=True) #mode
la_facility_pcua['HPSA_Score'].fillna(13.0, inplace=True) #mode,median 
la_facility_pcua['HPSA_PctPoverty'].fillna(26.0, inplace=True) #median
la_facility_pcua['HPSA_Ratio'] = la_facility_pcua['HPSA_Ratio'].str.replace(':1','')
la_facility_pcua['HPSA_Ratio'].fillna('5042', inplace=True) #mode
la_facility_pcua['HPSA_Ratio'] = la_facility_pcua['HPSA_Ratio'].astype('int64')
la_facility_pcua['HPSA_Pct_100Poverty'].fillna(26.42995285, inplace=True) #median
la_facility_pcua['HPSA_Pct_200Poverty'].fillna(56.74644013, inplace=True) #median
la_facility_pcua['license_category'].fillna('Community Clinic', inplace=True) #mode

la_facility_pcua['HPSA_Pct_100Poverty'] = la_facility_pcua['HPSA_Pct_100Poverty'].round(6)
la_facility_pcua['HPSA_Pct_200Poverty'] = la_facility_pcua['HPSA_Pct_200Poverty'].round(6)

In [6]:
la_facility_pcua['source'] = la_facility_pcua['source'].str.replace('211.0', '211')
la_facility_pcua['Zip Code'] = la_facility_pcua['Zip Code'].astype('int32')
la_facility_pcua = la_facility_pcua[la_facility_pcua['Zip Code']!=0]
la_facility_pcua['cat1+2'] = la_facility_pcua['cat2'].astype('str')+'_'+la_facility_pcua['cat1'].astype('str')

la_facility_pcua['city'] = la_facility_pcua['city'].str.replace('Signal Hill ', 'Signal Hill')
la_facility_pcua['city'] = la_facility_pcua['city'].str.replace(' Hollywoo', ' Hollywood').str.replace(' Hollywoodd', ' Hollywood')
la_facility_pcua['city'] = la_facility_pcua['city'].str.replace('Los Angeles,', 'Los Angeles').str.replace('Los Angeles ', 'Los Angeles').str.replace('Los Angelels','Los Angeles').str.replace('Los  Angeles', 'Los Angeles').str.replace(' Los Angeles', 'Los Angeles')
la_facility_pcua['city'] = la_facility_pcua['city'].str.replace('Industry', 'City of Industry').str.replace('City of Industry ', 'City of Industry').str.replace('City of City of Industry', 'City of Industry')
la_facility_pcua['city'] = la_facility_pcua['city'].str.replace('Bell', 'Bell Gardens').str.replace('Bell Gardens Gardens', 'Bell Gardens')

In [7]:
la_facility_pcua.columns

Index(['source', 'cat1', 'cat2', 'org_name', 'Name', 'addrln1', 'city',
       'post_id', 'description', 'link', 'latitude', 'longitude',
       'date_updated', 'Zip Code', 'facility_name', 'facility_description',
       'facility_status', 'license_category', 'facility_er_service',
       'facility_address_1', 'HPSA_RuralStatus', 'HPSA_MSSA_Name_1',
       'HPSA_MSSA_Name_2', 'HPSA_Subtype', 'HPSA_FTE', 'HPSA_Score',
       'HPSA_Designated_Time', 'HPSA_Updated_Time', 'HPSA_PctPoverty',
       'HPSA_Shortage', 'HPSA_Ratio', 'HPSA_CountPoverty',
       'HPSA_Pop_100Poverty', 'HPSA_Pop_200Poverty', 'HPSA_Pct_100Poverty',
       'HPSA_Pct_200Poverty', 'HPSA_UnitCount', 'cat1+2'],
      dtype='object')

### SideNote:

### 1) HPSA_Shortage: Higher Value -> Higher Primary Care Shortage

### 2) HPSA_Score (0-25): Higher Value -> Higher Primary Care Shortage

### 3) HPSA_Ratio: Lower Value -> higher shortage

### 4) HPSA_PctPoverty: Lower Percent -> Higher Poverty

### 5a) HPSA_Pct_100Poverty = Poverty Level Under 100%Poverty Threshold

### 5b) HPSA_Pct_100Poverty: Lower Percent -> Higher Poverty

### 6a) HPSA_Pct_200Poverty = Poverty Level Under 200%Poverty Threshold

### 6b) HPSA_Pct_200Poverty: Lower Percent -> Higher Poverty

### A) Comparisons of Shortage Scores, Poverty Level at ZipCode Levels

#### Let's look at 10 most frequent Zip Code Areas

In [8]:
la_facility_pcua['Zip Code'].value_counts().sort_values(ascending=False).head(10)

90022    663
93534    560
91606    540
90813    351
90640    264
90262    256
90011    230
93550    200
91331    200
90706    192
Name: Zip Code, dtype: int64

### 1) Primary Care Shortge Score of 10 Most Frequent Zip Code Areas

In [9]:
zip_score = {
    'HPSA_Score': ['mean']
}
zip_score = la_facility_pcua.groupby('Zip Code').agg(zip_score)
zip_score.columns = ['ByZip_' + ('_'.join(col).strip()) for col in zip_score.columns.values]
zip_score.reset_index(inplace=True)

In [10]:
zip_score.head(5)

,Zip Code,ByZip_HPSA_Score_mean
0,90001,17.0
1,90002,14.0
2,90003,15.0
3,90004,13.0
4,90005,13.0


In [11]:
topZip_HPSAscore = zip_score[(zip_score['Zip Code']==90022) | 
                            (zip_score['Zip Code']==93534) | 
                            (zip_score['Zip Code']==91606) | 
                            (zip_score['Zip Code']==90813) | 
                            (zip_score['Zip Code']==90640) |
                            (zip_score['Zip Code']==90262) | 
                            (zip_score['Zip Code']==90011) |
                            (zip_score['Zip Code']==93550) |
                            (zip_score['Zip Code']==91331) | 
                            (zip_score['Zip Code']==90706) 
                               ]

### -> Top 5 Highest Primary Care Shortages (by Most Frequent Zip Codes)
i) 90011 (15.500000)
 
ii) 90706 (15.000000)
 
iii) 93534 (15.000000)

iv) 90813 (14.444444)

v) 93550 (14.300000)

In [12]:
topZip_HPSAscore

,Zip Code,ByZip_HPSA_Score_mean
9,90011,15.500000
20,90022,11.769231
84,90262,4.875000
116,90640,12.590909
123,90706,15.000000
145,90813,14.444444
181,91331,14.125000
213,91606,13.000000
245,93534,15.000000
249,93550,14.300000


### 2) Poverty Level by Zip Codes

In [13]:
zip_poverty = {
    'HPSA_PctPoverty': ['mean']
}
zip_poverty = la_facility_pcua.groupby(['Zip Code']).agg(zip_poverty)
zip_poverty.columns = ['ByZip_' + ('_'.join(col).strip()) for col in zip_poverty.columns.values]
zip_poverty.reset_index(inplace=True)

In [14]:
zip_poverty.head(5)

,Zip Code,ByZip_HPSA_PctPoverty_mean
0,90001,41.200000
1,90002,37.500000
2,90003,36.833333
3,90004,29.100000
4,90005,26.000000


In [15]:
topZip_poverty = zip_poverty[(zip_poverty['Zip Code']==90022) | 
                            (zip_poverty['Zip Code']==93534) | 
                            (zip_poverty['Zip Code']==91606) | 
                            (zip_poverty['Zip Code']==90813) | 
                            (zip_poverty['Zip Code']==90640) |
                            (zip_poverty['Zip Code']==90262) | 
                            (zip_poverty['Zip Code']==90011) |
                            (zip_poverty['Zip Code']==93550) |
                            (zip_poverty['Zip Code']==91331) | 
                            (zip_poverty['Zip Code']==90706) 
                               ]

### -> Top 5 Highest Poverty Level (by most Freqeuent Zip Code Areas)
### (Lower Percent -> Higher Poverty)
i) 90706 (0.000000)

ii) 90640 (16.981818)

iii) 90262 (19.000000)

iv) 91331 (22.212500)

v) 91606 (23.900000)

### Concerned Area: 90706 (also in top 5 Highest Primary Care Shortage)

In [16]:
topZip_poverty

,Zip Code,ByZip_HPSA_PctPoverty_mean
9,90011,37.510000
20,90022,26.423077
84,90262,19.000000
116,90640,16.981818
123,90706,0.000000
145,90813,34.811111
181,91331,22.212500
213,91606,23.900000
245,93534,30.500000
249,93550,24.620000


> ### 3) Poverty Level Under 100% Poverty Threshold by Zip Codes

In [17]:
zip_100poverty = {
    'HPSA_Pct_100Poverty': ['mean']
}
zip_100poverty = la_facility_pcua.groupby(['Zip Code']).agg(zip_100poverty)
zip_100poverty.columns = ['ByZip_' + ('_'.join(col).strip()) for col in zip_100poverty.columns.values]
zip_100poverty.reset_index(inplace=True)

In [18]:
zip_100poverty.head(5)

,Zip Code,ByZip_HPSA_Pct_100Poverty_mean
0,90001,42.002689
1,90002,35.940386
2,90003,35.416117
3,90004,27.547487
4,90005,26.429953


In [19]:
topZip_100poverty = zip_100poverty[(zip_100poverty['Zip Code']==90022) | 
                            (zip_100poverty['Zip Code']==93534) | 
                            (zip_100poverty['Zip Code']==91606) | 
                            (zip_100poverty['Zip Code']==90813) | 
                            (zip_100poverty['Zip Code']==90640) |
                            (zip_100poverty['Zip Code']==90262) | 
                            (zip_100poverty['Zip Code']==90011) |
                            (zip_100poverty['Zip Code']==93550) |
                            (zip_100poverty['Zip Code']==91331) | 
                            (zip_100poverty['Zip Code']==90706) 
                               ]

### -> Top 5 Highest Poverty Level under 100%Threshold (by most Freqeuent Zip Code Areas)
### (Lower Percent -> Higher Poverty)
i) 90640 (16.040570)

ii) 90706 (18.938449)

iii) 90262 (19.298602)

iv) 91331 (21.880433)

v) 91606 (23.030476)

### Either Under 100% Threshold or general Poverty Level, these are same Zip Code areas with Highest Poverty Level
### Concerned Area: 90706 (also in both top 5 Highest Primary Care Shortage + top 5 Highest Poverty)

In [20]:
topZip_100poverty

,Zip Code,ByZip_HPSA_Pct_100Poverty_mean
9,90011,36.759326
20,90022,26.747578
84,90262,19.298602
116,90640,16.040570
123,90706,18.938449
145,90813,33.901961
181,91331,21.880433
213,91606,23.030476
245,93534,28.820426
249,93550,23.991426


### 4) Primary Care Shortage Score in Each City

In [21]:
city_score = {
    'HPSA_Score': ['mean']
}
city_score = la_facility_pcua.groupby('city').agg(city_score)
city_score.columns = ['ByCity_' + ('_'.join(col).strip()) for col in city_score.columns.values]
city_score.reset_index(inplace=True)

In [22]:
city_score.head(7)

,city,ByCity_HPSA_Score_mean
0,Acton,13.000
1,Agoura Hills,13.000
2,Alhambra,13.000
3,Altadena,13.000
4,Arcadia,13.000
5,Arleta,14.125
6,Artesia,13.000


In [23]:
topCity_HPSAscore = city_score[(city_score['city']=='Los Angeles') | 
                               (city_score['city']=='Hollywood') | 
                               (city_score['city']=='Pasadena') | 
                               (city_score['city']=='Long Beach') | 
                               (city_score['city']=='Pomona') |
                               (city_score['city']=='Santa Monica') | 
                               (city_score['city']=='Glendale') 
                               ]

### -> Top 5 Highest Primary Care Shortage (amongLA vs 6 Major Cities in county)

i) LA has 2nd Highest Shortage (14.064622)

ii) Long Beach has Highest Shortage (14.283495)

iii) Pomona has Lowest Shortage (10.016854)

### LA has 2nd worst shortage severity.

In [24]:
topCity_HPSAscore

,city,ByCity_HPSA_Score_mean
39,Glendale,13.000000
48,Hollywood,13.000000
64,Long Beach,14.283495
65,Los Angeles,14.064622
83,Pasadena,13.000000
85,Pomona,10.016854
99,Santa Monica,11.833333


### 5) Poverty Level in Each City

In [25]:
city_poverty = {
    'HPSA_PctPoverty': ['mean']
}
city_poverty = la_facility_pcua.groupby('city').agg(city_poverty)
city_poverty.columns = ['ByCity_' + ('_'.join(col).strip()) for col in city_poverty.columns.values]
city_poverty.reset_index(inplace=True)

In [26]:
city_poverty.head(5)

,city,ByCity_HPSA_PctPoverty_mean
0,Acton,26.0
1,Agoura Hills,26.0
2,Alhambra,26.0
3,Altadena,26.0
4,Arcadia,26.0


In [27]:
topCity_poverty = city_poverty[(city_poverty['city']=='Los Angeles') | 
                               (city_poverty['city']=='Hollywood') | 
                               (city_poverty['city']=='Pasadena') | 
                               (city_poverty['city']=='Long Beach') | 
                               (city_poverty['city']=='Pomona') |
                               (city_poverty['city']=='Santa Monica') | 
                               (city_poverty['city']=='Glendale') 
                               ]

### -> Top 5 Highest Poverty Level (among LA vs 6 Major Cities in county)
### -> Lower Value -> Higher Poverty

i) LA has 2nd Lowest Poverty Level (31.690690)

ii) Long Beach has Lowest Poverty Level (33.721748)

iii) Santa Monica has Highest Poverty Level (19.175000)


In [28]:
topCity_poverty

,city,ByCity_HPSA_PctPoverty_mean
39,Glendale,26.000000
48,Hollywood,29.100000
64,Long Beach,33.721748
65,Los Angeles,31.690690
83,Pasadena,26.000000
85,Pomona,25.204494
99,Santa Monica,19.175000


### 6) Poverty Level Under 100% Poverty Threshold in Each City

In [29]:
city_100poverty = {
    'HPSA_Pct_100Poverty': ['mean']
}
city_100poverty = la_facility_pcua.groupby('city').agg(city_100poverty)
city_100poverty.columns = ['ByCity_' + ('_'.join(col).strip()) for col in city_100poverty.columns.values]
city_100poverty.reset_index(inplace=True)

In [30]:
city_100poverty.head(5)

,city,ByCity_HPSA_Pct_100Poverty_mean
0,Acton,26.429953
1,Agoura Hills,26.429953
2,Alhambra,26.429953
3,Altadena,26.429953
4,Arcadia,26.429953


In [31]:
topCity_100poverty = city_100poverty[(city_100poverty['city']=='Los Angeles') | 
                               (city_100poverty['city']=='Hollywood') | 
                               (city_100poverty['city']=='Pasadena') | 
                               (city_100poverty['city']=='Long Beach') | 
                               (city_100poverty['city']=='Pomona') |
                               (city_100poverty['city']=='Santa Monica') | 
                               (city_100poverty['city']=='Glendale') 
                               ]

### -> Top 5 Highest Poverty Level Under 100%Threshold (among LA vs 6 Major Cities in county)

### -> Lower Value -> Higher Poverty

i) LA has 2nd Lowest Poverty Level (31.322811)

ii) Long Beach has Lowest Poverty Level (32.987687)

iii) Santa Monica has Highest Poverty Level (19.307832)

In [32]:
topCity_100poverty

,city,ByCity_HPSA_Pct_100Poverty_mean
39,Glendale,26.429953
48,Hollywood,27.547487
64,Long Beach,32.987687
65,Los Angeles,31.322811
83,Pasadena,26.429953
85,Pomona,24.377924
99,Santa Monica,19.307832


### 7) Poverty Level Under 200% Poverty Threshold in Each City

In [33]:
city_200poverty = {
    'HPSA_Pct_200Poverty': ['mean']
}
city_200poverty = la_facility_pcua.groupby('city').agg(city_200poverty)
city_200poverty.columns = ['ByCity_' + ('_'.join(col).strip()) for col in city_200poverty.columns.values]
city_200poverty.reset_index(inplace=True)

In [34]:
city_200poverty.head(5)

,city,ByCity_HPSA_Pct_200Poverty_mean
0,Acton,56.74644
1,Agoura Hills,56.74644
2,Alhambra,56.74644
3,Altadena,56.74644
4,Arcadia,56.74644


In [35]:
topCity_200poverty = city_200poverty[(city_200poverty['city']=='Los Angeles') | 
                               (city_200poverty['city']=='Hollywood') | 
                               (city_200poverty['city']=='Pasadena') | 
                               (city_200poverty['city']=='Long Beach') | 
                               (city_200poverty['city']=='Pomona') |
                               (city_200poverty['city']=='Santa Monica') | 
                               (city_200poverty['city']=='Glendale') 
                               ]

### -> Top 5 Highest Poverty Level Under 200%Threshold (among LA vs 6 Major Cities in county)

### -> Lower Value -> Higher Poverty

i) LA has 2nd Lowest Poverty Level (63.238785)

ii) Long Beach has Lowest Poverty Level (64.444871)

iii) Santa Monica has Highest Poverty Level (43.161866)

In [36]:
topCity_200poverty

,city,ByCity_HPSA_Pct_200Poverty_mean
39,Glendale,56.746440
48,Hollywood,57.238151
64,Long Beach,64.444871
65,Los Angeles,63.238785
83,Pasadena,56.746440
85,Pomona,55.892481
99,Santa Monica,43.161866


### 8) Primary Care Shorage in Each City by Healthcare & Community Services

In [37]:
city_services_score = {
    'HPSA_Score': ['mean']
}
city_services_score = la_facility_pcua.groupby(['city', 'cat1+2']).agg(city_services_score)
city_services_score.columns = ['ByCity+ServicesType_' + ('_'.join(col).strip()) for col in city_services_score.columns.values]
city_services_score.reset_index(inplace=True)

In [38]:
city_services_score.head(5)

,city,cat1+2,ByCity+ServicesType_HPSA_Score_mean
0,Acton,Substance Abuse Programs_Health and Mental Health,13.0
1,Agoura Hills,Domestic Violence Services_Social Services,13.0
2,Alhambra,Children and Family Services_Social Services,13.0
3,Alhambra,Crime Prevention and Support_Public Safety,13.0
4,Alhambra,Disability Support Services_Social Services,13.0


In [39]:
top5City_services_score = city_services_score[(city_services_score['city']=='Los Angeles') | 
                               (city_services_score['city']=='Hollywood') | 
                               (city_services_score['city']=='Pasadena') | 
                               (city_services_score['city']=='Long Beach') | 
                               (city_services_score['city']=='Santa Monica') 
                               ]

### -> Top 5 Highest Shortage (among LA vs 6 Major Cities in county)

i) LA has Highest Shortage Level  (14.451220) with Crime Reporting and Investigation_Public Safety

### which shows impacts of Healthcare Shortage on Community Development/Safety


In [40]:
top5City_services_score

,city,cat1+2,ByCity+ServicesType_HPSA_Score_mean
180,Hollywood,Children and Family Services_Social Services,13.000000
181,Hollywood,Crime Prevention and Support_Public Safety,13.000000
182,Hollywood,Crime Reporting and Investigation_Public Safety,13.000000
183,Hollywood,Domestic Violence Services_Social Services,13.000000
184,Hollywood,Mental Health Counseling_Health and Mental Health,13.000000
185,Hollywood,Substance Abuse Programs_Health and Mental Health,13.000000
240,Long Beach,Children and Family Services_Social Services,14.213333
241,Long Beach,Crime Prevention and Support_Public Safety,13.875000
242,Long Beach,Crime Reporting and Investigation_Public Safety,14.000000
243,Long Beach,Disability Support Services_Social Services,13.461538


### 9) Primary Care Shorage in Each City by Services License Type

In [41]:
city_license_score = {
    'HPSA_Score': ['mean']
}
city_license_score = la_facility_pcua.groupby(['city', 'license_category']).agg(city_license_score)
city_license_score.columns = ['ByCity+license_' + ('_'.join(col).strip()) for col in city_license_score.columns.values]
city_license_score.reset_index(inplace=True)

In [42]:
city_license_score.head(5)

,city,license_category,ByCity+license_HPSA_Score_mean
0,Acton,Community Clinic,13.0
1,Agoura Hills,Community Clinic,13.0
2,Alhambra,Community Clinic,13.0
3,Altadena,Community Clinic,13.0
4,Arcadia,Community Clinic,13.0


In [43]:
top5City_license_score = city_license_score[(city_license_score['city']=='Los Angeles') | 
                               (city_license_score['city']=='Hollywood') | 
                               (city_license_score['city']=='Pasadena') | 
                               (city_license_score['city']=='Long Beach') | 
                               (city_license_score['city']=='Santa Monica') 
                               ]

### -> LA has Highest Shortage Level (vs other 6 Major Cities in the County)
### -> LA's top 5 Shortage Category
i) Surgical Clinic
 
ii) Intermediate Care Facility

iii) Acute Psychiatric Hospital

iv) Skilled Nursing Facility

v) Chronic Dialysis Clinic 

In [44]:
top5City_license_score

,city,license_category,ByCity+license_HPSA_Score_mean
87,Hollywood,Community Clinic,13.000000
88,Hollywood,General Acute Care Hospital,13.000000
89,Hollywood,Home Health Agency,13.000000
90,Hollywood,Hospice,13.000000
122,Long Beach,Chemical Dep. Recovery Hospital,14.000000
123,Long Beach,Chronic Dialysis Clinic,14.000000
124,Long Beach,Community Clinic,14.546816
125,Long Beach,General Acute Care Hospital,14.000000
126,Long Beach,Psychology Clinic,14.000000
127,Long Beach,Skilled Nursing Facility,14.000000
